In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import tensorflow as tf
import numpy as np 
import cv2
import os
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import GlobalAveragePooling2D,Dropout,Convolution2D,Activation
from tensorflow.keras.utils import to_categorical

In [0]:
IMG_DIRECTORY = '/content/drive/My Drive/image_data'

CLASS_MAP = {
    "rock" : 0,
    "paper" : 1,
    "scissors" : 2,
    "none" : 3
}

NUM_CLASES = len(CLASS_MAP)

def mapper(val):
    return CLASS_MAP[val]


In [0]:
from tensorflow.keras.applications.densenet import DenseNet121

densent = DenseNet121(include_top=False, weights='imagenet',classes=4,input_shape=(227,227,3))

def get_model(base):
    
    model = Sequential()
    model.add(base)
    model.add(Dropout(0.5))
    model.add(Convolution2D(NUM_CLASES, (1,1), padding='valid'))
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))

    return model


In [0]:
dataset = []
counter = 0

for dir in os.listdir(IMG_DIRECTORY):
    counter = 0
    path = os.path.join(IMG_DIRECTORY, dir)
    for item in os.listdir(path):
      counter = counter + 1
      
      img = cv2.imread(os.path.join(path, item))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      img = cv2.resize(img, (227,227))
      counter = counter + 1 
      print(counter)
      
      dataset.append([img, dir])



In [0]:
print(len(dataset))

In [0]:
from random import shuffle
shuffle(dataset)

In [0]:

data, labels = zip(*dataset)
labels = list(map(mapper, labels))

labels = to_categorical(labels)

In [0]:
from tensorflow.keras.optimizers import Adam

dnet = get_model(densent)

dnet.compile(
    optimizer = Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['acc']
)


In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

np_train_data = np.array(data)
np_train_labels = np.array(labels)

checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True,
    mode='auto'
)

es = EarlyStopping(patience = 3)

dnet.fit(
    np_train_data,
    np_train_labels, 
    validation_split=0.2,
    verbose=2,
    epochs=10,
    callbacks=[checkpoint, es]
    )

dnet.save("model.h5")

Epoch 1/10

Epoch 00001: val_acc improved from -inf to 1.00000, saving model to model.h5
150/150 - 33s - loss: 0.0455 - acc: 0.9875 - val_loss: 0.0074 - val_acc: 1.0000
Epoch 2/10

Epoch 00002: val_acc did not improve from 1.00000
150/150 - 31s - loss: 5.6441e-04 - acc: 1.0000 - val_loss: 1.3404e-04 - val_acc: 1.0000
Epoch 3/10

Epoch 00003: val_acc did not improve from 1.00000
150/150 - 31s - loss: 3.3665e-04 - acc: 1.0000 - val_loss: 5.8799e-05 - val_acc: 1.0000
Epoch 4/10

Epoch 00004: val_acc did not improve from 1.00000
150/150 - 31s - loss: 1.5177e-04 - acc: 1.0000 - val_loss: 3.2377e-05 - val_acc: 1.0000
Epoch 5/10

Epoch 00005: val_acc did not improve from 1.00000
150/150 - 31s - loss: 1.1145e-04 - acc: 1.0000 - val_loss: 2.4955e-05 - val_acc: 1.0000
Epoch 6/10

Epoch 00006: val_acc did not improve from 1.00000
150/150 - 31s - loss: 1.0563e-04 - acc: 1.0000 - val_loss: 2.0145e-05 - val_acc: 1.0000
Epoch 7/10

Epoch 00007: val_acc did not improve from 1.00000
150/150 - 31s - los